# Importing Libraries

In [0]:
import warnings
warnings.filterwarnings("ignore")
import os
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Ridge,BayesianRidge,Lasso 
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score,precision_score,recall_score,mean_squared_error
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Mounting Drive

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

KeyError: ignored

In [0]:
!ls 'gdrive/My Drive/ASAG Work/All Data'

 elmo_semeval_embeddings.csv   numberbatch-en-17.02.txt
'Feature Selection.csv'        SemEval
'Feature Selection.gsheet'     SemEval_Cleaned.csv
 glove.6B.100d.txt	      'SemEval Full Data.csv'
 glove.6B.100d.txt.word2vec    SemEval_Question_Infersent_Embeddings.csv
 mini.h5		       SemEval_Ref_Ans_Infersent_Embeddings.csv
 Mohler1_ans.pickle	       SemEval_Stu_Ans_Infersent_Embeddings.csv
 Mohler1_ques.pickle	       SemEval_till_POS.csv
 MohlerFullData.csv	       SemEval_till_POSNew.csv
 Mohler_till_POS.csv	       ShortAnswerGrading_v1.0
 Mohler_till_POSNew.csv        ShortAnswerGrading_v2.0


## Loading Data

In [0]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_till_POSNew.csv",sep=',')

In [5]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'qid', 'ref_id', 'question', 'ref_answer',
       'stu_answer', 'train/test', 'accuracy', 'q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'Row_no', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 'how_flag', 'what_flag', 'why_flag',
       'who_flag', 'which_flag', 'when_flag', 'where_flag', 'wh

#Creating New Features

In [0]:
def fab(row, types):
  if(row['r_'+types]!=0):
    return row['s_'+types]/row['r_'+ types]
  else:
    return 0

In [0]:
data['s_verbs_vs_r_verbs'] = data.apply(lambda x : fab(x,"verbs"), axis = 1)
data['s_nouns_vs_r_nouns'] = data.apply(lambda x : fab(x,"nouns"), axis = 1)
data['s_adjectives_vs_r_adjectives'] = data.apply(lambda x : fab(x,"adjectives"), axis = 1)
data['s_word_count_vs_r_word_count'] = data['s_word_count']/data['r_word_count']

In [0]:
data['s_nouns_vs_words_vs_r_nouns_vs_words'] = data.apply(lambda x : fab(x, "nouns_vs_words"), axis = 1)
data['s_verbs_vs_words_vs_r_verbs_vs_words'] = data.apply(lambda x : fab(x, "verbs_vs_words"), axis = 1)
data['s_adjectives_vs_words_vs_r_adjectives_vs_words'] = data.apply(lambda x : fab(x, "adjectives_vs_words"), axis = 1)

##Choose which features and set classes

In [0]:
n_classes = 3

features = ['q_word_count', 'q_char_count',
       'q_avg_word', 'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word', 'q_nouns',
       'q_adjectives', 'q_verbs', 'q_nouns_vs_length',
       'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       'q_adjectives_vs_words', 'q_verbs_vs_words', 'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 's_verbs_vs_r_verbs','s_nouns_vs_r_nouns',
            's_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count' ]

features1 = [#'q_word_count', 'q_char_count', 'q_avg_word',
       'r_word_count', 'r_char_count', 'r_avg_word',
       's_word_count', 's_char_count', 's_avg_word',
       #'q_adjectives', 'q_verbs', 'q_nouns_vs_length', 'q_nouns'
       #'q_adjectives_vs_length', 'q_verbs_vs_length', 'q_nouns_vs_words',
       #'q_adjectives_vs_words', 'q_verbs_vs_words',
        'r_nouns', 'r_adjectives',
       'r_verbs', 'r_nouns_vs_length', 'r_adjectives_vs_length',
       'r_verbs_vs_length', 'r_nouns_vs_words', 'r_adjectives_vs_words',
       'r_verbs_vs_words', 's_nouns', 's_adjectives', 's_verbs',
       's_nouns_vs_length', 's_adjectives_vs_length', 's_verbs_vs_length',
       's_nouns_vs_words', 's_adjectives_vs_words', 's_verbs_vs_words',
       'precision', 'recall', 'F1_score', 's_verbs_vs_r_verbs','s_nouns_vs_r_nouns',
       's_adjectives_vs_r_adjectives','s_word_count_vs_r_word_count']
       #'how_flag', 'what_flag', 'why_flag',
       #'who_flag', 'which_flag', 'when_flag', 'where_flag', 'whom_flag','accuracy']

label = 'accuracy'

In [0]:
def get_req_data(data,features,label,tr_ts,n_class,q_embed=None,r_embed=None,s_embed=None):
    indices = data[(data['train/test'].str.contains(tr_ts)) & (data['train/test'].str.contains(str(n_class)))].index.tolist()
    d = data.loc[indices,:].reset_index(drop=True)
    f = d.loc[:,features]
    y = d.loc[:,label]
    if q_embed == None:
        return f,y
    else:
        q = q_embed.iloc[indices,:].reset_index(drop=True)
        r = r_embed.iloc[indices,:].reset_index(drop=True)
        s = s_embed.iloc[indices,:].reset_index(drop=True)
        return f,q,r,s,y        

In [0]:
# run if no embeddings loaded
f_train,y_train = get_req_data(data,features1,label,'train',n_classes)
f_test1,y_test1 = get_req_data(data,features1,label,'unseen-answers',n_classes)
f_test2,y_test2 = get_req_data(data,features1,label,'unseen-questions',n_classes)
f_test3,y_test3 = get_req_data(data,features1,label,'unseen-domains',n_classes)

In [0]:
q_embed = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Question_Infersent_Embeddings.csv",sep=',')
r_embed = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Ref_Ans_Infersent_Embeddings.csv",sep=',')
s_embed = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval_Stu_Ans_Infersent_Embeddings.csv",sep=',')

In [0]:
# run only if embeddings loaded
f_train,q_train,r_train,s_train,y_train = get_req_data(data,features,label,'train',n_classes,q_embed,r_embed,s_embed)
f_test1,q_test1,r_test1,s_test1,y_test1 = get_req_data(data,features,label,'unseen-answers',n_classes,q_embed,r_embed,s_embed)
f_test2,q_test2,r_test2,s_test2,y_test2 = get_req_data(data,features,label,'unseen-questions',n_classes,q_embed,r_embed,s_embed)
f_test3,q_test3,r_test3,s_test3,y_test3 = get_req_data(data,features,label,'unseen-domains',n_classes,q_embed,r_embed,s_embed)

In [0]:
# convert labels to ids
un_acc = y_train.unique()
lab_to_int, int_to_lab = {},{}
for n,u in enumerate(un_acc):
    lab_to_int[u] = n
    int_to_lab[n] = u
    
y_train = y_train.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
y_test1 = y_test1.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
y_test2 = y_test2.replace(list(int_to_lab.values()), list(int_to_lab.keys()))
y_test3 = y_test3.replace(list(int_to_lab.values()), list(int_to_lab.keys()))

# Mohler

In [0]:
data = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/Mohler_till_POSNew.csv",sep=',')

In [0]:
features = ['Q_word_count', 'Q_char_count', 'Q_avg_word', 'R_word_count',
       'R_char_count', 'R_avg_word', 'A_word_count', 'A_char_count',
       'A_avg_word', 'Q_nouns', 'Q_adjectives', 'Q_verbs',
       'Q_nouns_vs_length', 'Q_adjectives_vs_length', 'Q_verbs_vs_length',
       'Q_nouns_vs_words', 'Q_adjectives_vs_words', 'Q_verbs_vs_words',
       'R_nouns', 'R_adjectives', 'R_verbs', 'R_nouns_vs_length',
       'R_adjectives_vs_length', 'R_verbs_vs_length', 'R_nouns_vs_words',
       'R_adjectives_vs_words', 'R_verbs_vs_words', 'A_nouns', 'A_adjectives',
       'A_verbs', 'A_nouns_vs_length', 'A_adjectives_vs_length',
       'A_verbs_vs_length', 'A_nouns_vs_words', 'A_adjectives_vs_words',
       'A_verbs_vs_words', 'precision', 'recall', 'F1_score', 'how_flag',
       'what_flag', 'why_flag', 'who_flag', 'which_flag', 'when_flag',
       'where_flag', 'whom_flag']

label = 'Score'

In [0]:
split = int(0.7*data.shape[0])
f_train,f_test = data.loc[:split,features], data.loc[split:,features]
y_train,y_test = data.loc[:split,label], data.loc[split:,label]

In [0]:
breg = BayesianRidge(normalize=True)
rfr = RandomForestRegressor(n_estimators=500,criterion='mse')
etr = ExtraTreesRegressor(n_estimators=500,criterion='mse')
gbr = GradientBoostingRegressor(n_estimators=500,loss='huber',learning_rate=0.1)

In [0]:
breg.fit(f_train,y_train)
rfr.fit(f_train,y_train)
etr.fit(f_train,y_train)
gbr.fit(f_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [0]:
y_pred = breg.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = rfr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = etr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))
y_pred = gbr.predict(f_test)
print("RMSE : %.3f \t Corr : %.3f"%(np.sqrt(mean_squared_error(y_pred,y_test)),np.corrcoef(y_test,y_pred)[0,1]))

RMSE : 0.982 	 Corr : 0.545
RMSE : 0.715 	 Corr : 0.802
RMSE : 0.644 	 Corr : 0.842
RMSE : 0.780 	 Corr : 0.748


# Tree Models

In [0]:
def run_tree(model,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,method,to3=False):
    results = []

    y_pred = model.predict(f_test1)
    if n_classes == 2:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    y_pred = model.predict(f_test2)
    if n_classes == 2:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])

    y_pred = model.predict(f_test3)
    if n_classes == 2:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    return (results[2],results[5],results[8])
    return results

## Normal Run

In [0]:
'''
Use 5way model for 3way
0,3 correct
1 contradict
2,4 incorrect
'''
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
rfc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'weighted',True))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'macro',True))

-------Weighted--------------
[0.6279719024930098, 0.5617977528089888, 0.5229124062674932, 0.5524530846476651, 0.5032216494845361, 0.44187471324368516, 0.5734580832225215, 0.5475668566418238, 0.49467921174417345]
-------Macro--------------
[0.6290520156185516, 0.5092583135410992, 0.4998419828789838, 0.576474066410746, 0.42278473339544115, 0.38495291709397467, 0.4145592076688261, 0.41654374682164397, 0.36769054639984994]


In [77]:
rfc = RandomForestClassifier(n_estimators=300,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=100,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=100)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))

-------Weighted--------------
(0.7879828326180257, 0.771859296482412, 0.7575913535769428)
(0.7880910683012259, 0.7627718765806778, 0.748765116675183)
(0.7915611814345992, 0.7667342799188641, 0.7477128782547502)
-------Macro--------------
(0.7879828326180257, 0.771859296482412, 0.7575913535769428)
(0.7880910683012259, 0.7627718765806778, 0.748765116675183)
(0.7915611814345992, 0.7667342799188641, 0.7477128782547502)


In [0]:
rfc = RandomForestClassifier(n_estimators=300,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=100,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=100)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))

In [0]:
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=1000)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,2,'macro'))

-------Weighted--------------
[0.7861952861952862, 0.8192982456140351, 0.8024054982817869, 0.7069124423963133, 0.8456449834619625, 0.7700803212851405, 0.6962104603820858, 0.8404536862003781, 0.7615621788283657]
[0.7864583333333334, 0.7947368421052632, 0.7905759162303665, 0.693822739480752, 0.8544652701212789, 0.7658102766798418, 0.6799881586737715, 0.8684310018903592, 0.7627428191930932]
[0.7779661016949152, 0.8052631578947368, 0.7913793103448276, 0.7328405491024287, 0.7651598676957001, 0.7486515641855448, 0.7076399394856279, 0.7073724007561437, 0.7075061448288902]
-------Macro--------------
[0.7809847198641766, 0.8070175438596491, 0.7937877480586712, 0.7068493150684931, 0.8533627342888643, 0.7732267732267732, 0.6935734244023595, 0.8446124763705104, 0.7616774633481076]
[0.7885615251299827, 0.7982456140350878, 0.7933740191804708, 0.6946428571428571, 0.8577728776185226, 0.7676369018253577, 0.682962962962963, 0.8714555765595463, 0.7657807308970098]
[0.7766497461928934, 0.8052631578947368,

In [105]:
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=1000)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,3,'macro'))

-------Weighted--------------
(0.657520108074168, 0.5400191405081427, 0.580394732502212)
(0.6376174102076422, 0.5424865024250181, 0.5871461610612861)
(0.6375421851492873, 0.5157878968370408, 0.5744252133806499)
-------Macro--------------
(0.6199029247235118, 0.48527101314460946, 0.42704779623985073)
(0.6012436504895026, 0.4916020687207127, 0.42960025315966516)
(0.6056547407414555, 0.46200351607890694, 0.43630920604121065)


In [0]:
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=200)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))

-------Weighted--------------
(0.5934057265826816, 0.4954611940931806, 0.43850426865706926)
(0.5773718437467306, 0.46184321808048123, 0.45275813579016544)
(0.5555228321910722, 0.48257699259618253, 0.46186830119994743)
-------Macro--------------
(0.5798881827794646, 0.49218718320508853, 0.30193458384430777)
(0.5518208627683302, 0.4538291544821975, 0.276951090667672)
(0.5268584461895788, 0.46475107440896296, 0.3668793341597022)


In [0]:
rfc = RandomForestClassifier(n_estimators=500,criterion='entropy')
etc = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
gbc = GradientBoostingClassifier(loss='deviance',learning_rate=0.1,n_estimators=1000)
rfc.fit(f_train,y_train)
etc.fit(f_train,y_train)
gbc.fit(f_train,y_train)
print("-------Weighted--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'weighted'))
print("-------Macro--------------")
print(run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))
print(run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))
print(run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,5,'macro'))

-------Weighted--------------
[0.5890202872294458, 0.5944841675178754, 0.5889878184840975, 0.5218586531391214, 0.5335051546391752, 0.5114656067578713, 0.43739159116932247, 0.47983340640070143, 0.4487120761441993]
[0.5784639675463775, 0.5832482124616956, 0.578739571663111, 0.4875519157143638, 0.49677835051546393, 0.47004041651986606, 0.43116269750571556, 0.4907935116177115, 0.45109472266160766]
[0.5761674268501822, 0.5801838610827375, 0.5743280416650655, 0.4900321192960472, 0.4948453608247423, 0.4805899318790853, 0.43660495874430777, 0.46536606751424814, 0.44481848668076857]
-------Macro--------------
[0.6074086980906613, 0.563972274063679, 0.582101544155204, 0.5243207673971076, 0.4965476932903556, 0.4962569916339465, 0.32885202048114154, 0.3246657724490758, 0.3180543664699963]
[0.6001522114076996, 0.5335494053101726, 0.55966081598323, 0.4860289763963307, 0.43482699460754953, 0.4411386275703566, 0.2930028415980131, 0.3004683245129444, 0.2899753507163193]
[0.5984138141077756, 0.540889004

## Grid Run

def run_grid(mtype,grid,n_estimators,criterion,max_depth,max_features,max_leaf_nodes,learning_rate,
            f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes):
    counter = 0
    for est in n_estimators:
        for md in max_depth:
            for mf in max_features:
                for ml in max_leaf_nodes:
                    if mtype == 'RFC':
                        for crt in criterion:
                            counter+=1
                            rfc = RandomForestClassifier(n_estimators=est,criterion=crt,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            rfc.fit(f_train,y_train)
                            res = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'RFC',
                                   'n_estimators':int(est),
                                   'criterion':str(crt),
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':None,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%20 == 0:
                                print(counter)
                    elif mtype == 'ETC':
                        for crt in criterion:
                            counter+=1
                            etc = ExtraTreesClassifier(n_estimators=est,criterion=crt,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            etc.fit(f_train,y_train)
                            res = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'ETC',
                                   'n_estimators':int(est),
                                   'criterion':str(crt),
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':None,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%20 == 0:
                                print(counter)
                    else:
                        for lr in learning_rate:
                            counter+=1
                            gbc = GradientBoostingClassifier(n_estimators=est,loss='deviance',learning_rate=lr,max_depth=md,max_features=mf,max_leaf_nodes=ml)
                            gbc.fit(f_train,y_train)
                            res = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                            res1 = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                            tot = {'model':'GBC',
                                   'n_estimators':int(est),
                                   'criterion':None,
                                   'max_depth':int(md),
                                   'max_features':str(mf),
                                   'max_leaf_nodes':int(ml),
                                   'learning_rate':lr,
                                   'UA_m_pr':res[0],
                                   'UA_m_re':res[1],
                                   'UA_m_f1':res[2],
                                   'UQ_m_pr':res[3],
                                   'UQ_m_re':res[4],
                                   'UQ_m_f1':res[5],
                                   'UD_m_pr':res[6],
                                   'UD_m_re':res[7],
                                   'UD_m_f1':res[8],
                                   'UA_w_pr':res1[0],
                                   'UA_w_re':res1[1],
                                   'UA_w_f1':res1[2],
                                   'UQ_w_pr':res1[3],
                                   'UQ_w_re':res1[4],
                                   'UQ_w_f1':res1[5],
                                   'UD_w_pr':res1[6],
                                   'UD_w_re':res1[7],
                                   'UD_w_f1':res1[8]
                                  }
                            grid = grid.append(tot,ignore_index=True)
                            if counter%50 == 0:
                                print(counter)
    return grid

n_estimators = [100,200,300,400,500]
criterion = ['gini','entropy']
max_depth = [5,10,15,20,25,30]
max_features = ['sqrt','log2']
max_leaf_nodes = [10,25,50,75]
learning_rate = [0.1,0.3,0.01]

grid = pd.DataFrame([],columns=['model','n_estimators','criterion','learning_rate',
                                'UA_m_pr','UA_m_re','UA_m_f1','UQ_m_pr','UQ_m_re','UQ_m_f1','UD_m_pr','UD_m_re','UD_m_f1',
                                'UA_w_pr','UA_w_re','UA_w_f1','UQ_w_pr','UQ_w_re','UQ_w_f1','UD_w_pr','UD_w_re','UD_w_f1'])

In [0]:
def run_grid1(mtype,grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes):
    counter = 0
    for est in n_estimators:
        if mtype == 'RFC':
            for crt in criterion:
                counter+=1
                print(counter)
                rfc = RandomForestClassifier(n_estimators=est,criterion=crt)
                rfc.fit(f_train,y_train)
                res = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(rfc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'RFC',
                       'n_estimators':int(est),
                       'criterion':str(crt),
                       'learning_rate':None,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)

        elif mtype == 'ETC':
            for crt in criterion:
                counter+=1
                print(counter)
                etc = ExtraTreesClassifier(n_estimators=est,criterion=crt)
                etc.fit(f_train,y_train)
                res = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(etc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'ETC',
                       'n_estimators':int(est),
                       'criterion':str(crt),
                       'learning_rate':None,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)

        else:
            for lr in learning_rate:
                counter+=1
                print(counter)
                gbc = GradientBoostingClassifier(n_estimators=est,loss='deviance',learning_rate=lr)
                gbc.fit(f_train,y_train)
                res = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'macro')
                res1 = run_tree(gbc,f_test1,f_test2,f_test3,y_test1,y_test2,y_test3,n_classes,'weighted')
                tot = {'model':'GBC',
                       'n_estimators':int(est),
                       'criterion':None,
                       'learning_rate':lr,
                       'UA_m_pr':res[0],
                       'UA_m_re':res[1],
                       'UA_m_f1':res[2],
                       'UQ_m_pr':res[3],
                       'UQ_m_re':res[4],
                       'UQ_m_f1':res[5],
                       'UD_m_pr':res[6],
                       'UD_m_re':res[7],
                       'UD_m_f1':res[8],
                       'UA_w_pr':res1[0],
                       'UA_w_re':res1[1],
                       'UA_w_f1':res1[2],
                       'UQ_w_pr':res1[3],
                       'UQ_w_re':res1[4],
                       'UQ_w_f1':res1[5],
                       'UD_w_pr':res1[6],
                       'UD_w_re':res1[7],
                       'UD_w_f1':res1[8]
                      }
                grid = grid.append(tot,ignore_index=True)
             
    return grid

In [0]:
n_estimators = [100,200,300,400,500]
criterion = ['gini','entropy']
learning_rate = [0.1,0.3,0.01]

# grid = pd.DataFrame([],columns=['model','n_estimators','criterion','learning_rate',
#                                 'UA_m_pr','UA_m_re','UA_m_f1','UQ_m_pr','UQ_m_re','UQ_m_f1','UD_m_pr','UD_m_re','UD_m_f1',
#                                 'UA_w_pr','UA_w_re','UA_w_f1','UQ_w_pr','UQ_w_re','UQ_w_f1','UD_w_pr','UD_w_re','UD_w_f1'])

In [0]:
grid = run_grid1('RFC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+n_classes+"way_tree_grid_search.csv",sep=',',index=None)
grid = run_grid1('ETC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+n_classes+"way_tree_grid_search.csv",sep=',',index=None)
grid = run_grid1('GBC',grid,n_estimators,criterion,learning_rate,f_train,f_test1,f_test2,f_test3,y_train,y_test1,y_test2,y_test3,n_classes)
grid.to_csv("gdrive/My Drive/ASAG Work/Results/"+n_classes+"way_tree_grid_search.csv",sep=',',index=None)

In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/5way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:5,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
24,RFC,500,entropy,NaN,0.582376,0.592112,0.496443,0.505764,0.311870,0.441613
22,RFC,400,entropy,NaN,0.582196,0.597657,0.508901,0.520132,0.315043,0.440064
23,RFC,500,gini,NaN,0.578844,0.591909,0.514204,0.517117,0.320927,0.446049
17,RFC,200,gini,NaN,0.577102,0.593578,0.506222,0.514550,0.313085,0.438474
19,RFC,300,gini,NaN,0.575516,0.589647,0.513223,0.519979,0.303533,0.441129
20,RFC,300,entropy,NaN,0.573535,0.593533,0.515281,0.524838,0.317814,0.445992
18,RFC,200,entropy,NaN,0.572964,0.591256,0.509757,0.512090,0.325381,0.444656
12,GBC,500,NaN,0.1,0.569709,0.576469,0.464728,0.478279,0.316063,0.447306
16,RFC,100,entropy,NaN,0.568882,0.585556,0.499496,0.503325,0.329174,0.444705
21,RFC,400,gini,NaN,0.565498,0.588946,0.510506,0.522731,0.308816,0.443785


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
3,GBC,200,NaN,0.10,0.533163,0.556563,0.459228,0.481210,0.364422,0.461961
11,GBC,400,NaN,0.01,0.482715,0.513551,0.471959,0.482693,0.359993,0.451158
8,GBC,300,NaN,0.01,0.462408,0.493590,0.445488,0.453131,0.359932,0.447367
14,GBC,500,NaN,0.01,0.491824,0.527743,0.466784,0.476357,0.357434,0.457577
0,GBC,100,NaN,0.10,0.517108,0.540609,0.451802,0.479523,0.341750,0.453206
1,GBC,100,NaN,0.30,0.544924,0.570197,0.451995,0.471771,0.341374,0.459929
6,GBC,300,NaN,0.10,0.561324,0.570076,0.460910,0.477263,0.340614,0.455058
9,GBC,400,NaN,0.10,0.558849,0.572732,0.460966,0.475082,0.336737,0.456677
16,RFC,100,entropy,NaN,0.568882,0.585556,0.499496,0.503325,0.329174,0.444705
4,GBC,200,NaN,0.30,0.546170,0.573552,0.460454,0.479169,0.328757,0.453452


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/3way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
6,RFC,400,gini,NaN,0.631032,0.663615,0.507533,0.556199,0.435268,0.593633
3,RFC,200,entropy,NaN,0.626781,0.657946,0.497607,0.550042,0.430351,0.587169
2,RFC,200,gini,NaN,0.626515,0.660540,0.502527,0.552949,0.428694,0.584765
7,RFC,400,entropy,NaN,0.624369,0.658787,0.495816,0.549027,0.435443,0.593913
0,RFC,100,gini,NaN,0.623984,0.658715,0.496428,0.547091,0.431081,0.587824
5,RFC,300,entropy,NaN,0.622783,0.655971,0.495136,0.548389,0.434387,0.592597
9,RFC,500,entropy,NaN,0.622328,0.657895,0.485880,0.540578,0.437756,0.594161
4,RFC,300,gini,NaN,0.619829,0.654432,0.495905,0.548085,0.439123,0.598578
11,ETC,100,entropy,NaN,0.616931,0.651503,0.494389,0.542584,0.434433,0.594401
8,RFC,500,gini,NaN,0.616885,0.653341,0.505867,0.556426,0.429571,0.585732


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
20,GBC,100,NaN,0.10,0.572167,0.615352,0.490411,0.547519,0.442560,0.589050
31,GBC,400,NaN,0.01,0.523924,0.588001,0.417325,0.498173,0.440172,0.590715
4,RFC,300,gini,NaN,0.619829,0.654432,0.495905,0.548085,0.439123,0.598578
28,GBC,300,NaN,0.01,0.505613,0.575037,0.412324,0.495920,0.438645,0.588913
34,GBC,500,NaN,0.01,0.536036,0.593903,0.426078,0.502815,0.438594,0.589848
18,ETC,500,gini,NaN,0.606863,0.642934,0.483198,0.533370,0.438496,0.600284
23,GBC,200,NaN,0.10,0.587885,0.628948,0.495741,0.550982,0.437842,0.584860
9,RFC,500,entropy,NaN,0.622328,0.657895,0.485880,0.540578,0.437756,0.594161
12,ETC,200,gini,NaN,0.602257,0.638784,0.483525,0.533304,0.437208,0.598490
19,ETC,500,entropy,NaN,0.610607,0.646342,0.476759,0.530369,0.436943,0.598406


In [0]:
grid = pd.read_csv("gdrive/My Drive/ASAG Work/Results/2way_tree_grid_search.csv",sep=',')

In [0]:
g1 = grid.sort_values(by=['UA_m_f1','UA_w_f1','UQ_m_f1','UQ_w_f1','UD_m_f1','UD_w_f1'],ascending=False)
g1.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
5,RFC,300,entropy,NaN,0.798629,0.798629,0.769770,0.769770,0.759693,0.759693
3,RFC,200,entropy,NaN,0.798617,0.798617,0.773679,0.773679,0.760506,0.760506
6,RFC,400,gini,NaN,0.796217,0.796217,0.768844,0.768844,0.761529,0.761529
7,RFC,400,entropy,NaN,0.795160,0.795160,0.773253,0.773253,0.760226,0.760226
32,GBC,500,NaN,0.1,0.794850,0.794850,0.752284,0.752284,0.716798,0.716798
8,RFC,500,gini,NaN,0.794449,0.794449,0.776376,0.776376,0.758941,0.758941
29,GBC,400,NaN,0.1,0.793814,0.793814,0.761290,0.761290,0.721245,0.721245
2,RFC,200,gini,NaN,0.793761,0.793761,0.778057,0.778057,0.758938,0.758938
0,RFC,100,gini,NaN,0.792808,0.792808,0.775449,0.775449,0.752169,0.752169
12,ETC,200,gini,NaN,0.791557,0.791557,0.767096,0.767096,0.767327,0.767327


In [0]:
g2 = grid.sort_values(by=['UD_m_f1','UD_w_f1','UQ_m_f1','UQ_w_f1','UA_m_f1','UA_w_f1'],ascending=False)
g2.iloc[:20,[0,1,2,3,6,15,9,18,12,21]]

,model,n_estimators,criterion,learning_rate,UA_m_f1,UA_w_f1,UQ_m_f1,UQ_w_f1,UD_m_f1,UD_w_f1
12,ETC,200,gini,NaN,0.791557,0.791557,0.767096,0.767096,0.767327,0.767327
9,RFC,500,entropy,NaN,0.789700,0.789700,0.775984,0.775984,0.765487,0.765487
17,ETC,400,entropy,NaN,0.789199,0.789199,0.765321,0.765321,0.764599,0.764599
14,ETC,300,gini,NaN,0.787719,0.787719,0.770370,0.770370,0.764323,0.764323
13,ETC,200,entropy,NaN,0.782456,0.782456,0.775248,0.775248,0.764023,0.764023
19,ETC,500,entropy,NaN,0.790576,0.790576,0.766420,0.766420,0.763691,0.763691
15,ETC,300,entropy,NaN,0.790252,0.790252,0.769307,0.769307,0.763127,0.763127
16,ETC,400,gini,NaN,0.790576,0.790576,0.770069,0.770069,0.762900,0.762900
6,RFC,400,gini,NaN,0.796217,0.796217,0.768844,0.768844,0.761529,0.761529
22,GBC,100,NaN,0.01,0.762490,0.762490,0.765069,0.765069,0.761453,0.761453


# NN model

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

In [0]:
def apply_normalize(f,method=0):
    if method == 0:
        # min max
        f = (f-f.min())/(f.max()-f.min())
    elif method == 1:
        # z score
        f = (f-f.mean())/f.std()
    return f

In [0]:
def build_keras_dnn(var,units,act,drop,regu,regu_val):
    
    if regu == 'l1':
        re = tf.keras.regularizers.l1(regu_val)
    elif regu == 'l2':
        re = tf.keras.regularizers.l2(regu_val)
    else:
        re = tf.keras.regularizers.l1_l2(l1=regu_val,l2=regu_val)
    
    inp1 = tf.keras.layers.Input((len(features),))
    q_embed = tf.keras.layers.Input((4096,))
    r_embed = tf.keras.layers.Input((4096,))
    s_embed = tf.keras.layers.Input((4096,))
    
#     q_embed = tf.keras.layers.Dense(300)(qinp)
#     r_embed = tf.keras.layers.Dense(300)(rinp)
#     s_embed = tf.keras.layers.Dense(300)(sinp)
    
    add_qr = tf.keras.layers.Add()([q_embed,r_embed])
    add_rs = tf.keras.layers.Add()([r_embed,s_embed])
    sub_rs = tf.keras.layers.Subtract()([s_embed,r_embed])
    sub_srq = tf.keras.layers.Subtract()([s_embed,add_qr])
    mul_rs = tf.keras.layers.Multiply()([r_embed,s_embed])
    
    if var == 1:
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs,inp1])

        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
    
    if var == 2:
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(inp1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs])
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        
        concat = tf.keras.layers.Concatenate()([x1,x2])
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        x = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x)
        x = tf.keras.layers.Dropout(drop)(x)
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
        
    if var == 3:
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(inp1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x1)
        x1 = tf.keras.layers.Dropout(drop)(x1)
        x1 = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x1)
        
        concat = tf.keras.layers.Concatenate()([add_qr,add_rs,sub_rs,sub_srq,mul_rs])
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(concat)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(units,activation=act,activity_regularizer=re)(x2)
        x2 = tf.keras.layers.Dropout(drop)(x2)
        x2 = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x2)
        
        x = tf.keras.layers.Multiply()([x1,x2])
        out = tf.keras.layers.Dense(len(un_acc),activation='softmax')(x)
        
    model = tf.keras.models.Model(inputs=[inp1,q_embed,r_embed,s_embed],outputs=[out],name='DNN_Embed_Features')
    return model

## Normal Run

In [0]:
def run_model(norm_val,var,f_train,q_train,r_train,s_train,y_train):
    
#     feat_train = apply_normalize(f_train,norm_val)
    feat_train = f_train
    keras_model = build_keras_dnn(var,64,'relu',0.2,'l1',0.01)
    tf.keras.backend.clear_session()
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                                  strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

    tpu_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tpu_model.fit([feat_train,q_train,r_train,s_train],y_train,epochs=25,batch_size=64,callbacks=[tf.keras.callbacks.EarlyStopping('loss',0.1,10)])
    tpu_model.save_weights('./model.h5', overwrite=True)

In [0]:
def collect_results(norm_val,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,n_classes,method):

    model = build_keras_dnn(var,64,'relu',0.2,'l1',0.01)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    model.load_weights("./model.h5")

#     feat_test1 = apply_normalize(f_test1,norm_val)
#     feat_test2 = apply_normalize(f_test2,norm_val)
#     feat_test3 = apply_normalize(f_test3,norm_val)
    feat_test1 = f_test1
    feat_test2 = f_test2
    feat_test3 = f_test3
    
    results = []
    probs = model.predict([feat_test1,q_test1,r_test1,s_test1])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test1,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict([feat_test2,q_test2,r_test2,s_test2])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test2,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    probs = model.predict([feat_test3,q_test3,r_test3,s_test3])
    y_pred = np.argmax(probs,axis=1)
    if n_classes == 2:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average='binary')
    else:
        f1 = f1_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        pr = precision_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
        re = recall_score(y_test3,y_pred,labels=list(int_to_lab.keys()),average=method)
    results.extend([pr,re,f1])
    
    return results

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 1
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 2
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,2,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,3,'macro'))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.30.221.218:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 488913527654577071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5059204062510977768)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4866884899549682779)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12044553498752158692)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8907160912857058006)
INFO:tensorflow:*** Available Device: _DeviceA

In [0]:
var = 3
norm = 0
run_model(norm,var,f_train,q_train,r_train,s_train,y_train)
print("-------Weighted--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'weighted'))
print("-------Macro--------------")
print(collect_results(norm,var,f_test1,q_test1,r_test1,s_test1,y_test1,f_test2,q_test2,r_test2,s_test2,y_test2,
                    f_test3,q_test3,r_test3,s_test3,y_test3,5,'macro'))

## Grid Run

In [0]:
def grid_nn(normalize,variant,activation,units,dropout,regularize,regu_value,learning_rate,optimizer,grid,
           q_train,r_train,s_train,feat_train,y_train,q_test1,r_test1,s_test1,feat_test1,y_test1,
            q_test2,r_test2,s_test2,feat_test2,y_test2,q_test3,r_test3,s_test3,feat_test3,y_test3):
    for norm in normalize:
        for var in variant:
            for act in activation:
                for u in units:
                    for d in dropout:
                        for r in regularize:
                            for rv in regu_value:
                                for lr in learning_rate:
                                    for opt in optimizer:
                                        if opt == 'adam':
                                            optim = tf.train.AdamOptimizer(learning_rate=lr)
                                        elif opt == 'sgd':
                                            optim = tf.train.GradientDescentOptimizer(learning_rate=lr)
                                        elif opt == 'adadelta':
                                            optim = tf.train.AdadeltaOptimizer(learning_rate=lr)
                                        elif opt == 'adagrad':
                                            optim = tf.train.AdagradOptimizer(learning_rate=lr)
                                        elif opt == 'rmsprop':
                                            optim = tf.train.RMSPropOptimizer(learning_rate=lr)
                                        feat_train = apply_normalize(f_train,norm)
                                        feat_test1 = apply_normalize(f_test1,norm)
                                        feat_test2 = apply_normalize(f_test2,norm)
                                        feat_test3 = apply_normalize(f_test3,norm)
                                        keras_model = build_keras_dnn(var,u,act,d,r,rv)
                                        keras_model.summary()
                                        tf.keras.backend.clear_session()
                                        tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                                                                      strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                                                          tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

                                        tpu_model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
                                        callback = tf.keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.3,patience=3)
                                        tpu_model.fit([feat_train,q_train,r_train,s_train],y_train,epochs=25,batch_size=64,callbacks=[callback])
                                        tpu_model.save_weights('./model.h5', overwrite=True)
                                        # results
                                        res = []
                                        model = build_keras_dnn(var)
                                        model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
                                        model.load_weights("./model.h5")
                                        #UA
                                        probs = model.predict([feat_test1,q_test1,r_test1,s_test1])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test1,y_pred)
                                        pr = precision_score(y_test1,y_pred)
                                        re = recall_score(y_test1,y_pred)
                                        res.extend([pr,re,f1])
                                        #UQ
                                        probs = model.predict([feat_test2,q_test2,r_test2,s_test2])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test2,y_pred)
                                        pr = precision_score(y_test2,y_pred)
                                        re = recall_score(y_test2,y_pred)
                                        res.extend([pr,re,f1])
                                        #UD
                                        probs = model.predict([feat_test3,q_test3,r_test3,s_test3])
                                        y_pred = np.argmax(probs,axis=1)
                                        f1 = f1_score(y_test3,y_pred)
                                        pr = precision_score(y_test3,y_pred)
                                        re = recall_score(y_test3,y_pred)
                                        res.extend([pr,re,f1])
                                        tot = {'variant':var,
                                               'normalize':norm,
                                               'activation':act,
                                               'units':u,
                                               'dropout':d,
                                               'regularizer':r,
                                               'regularizer_value':rv,
                                               'learning_rate':lr,
                                               'optimizer':opt,
                                               'UA_pr':res[0],
                                               'UA_re':res[1],
                                               'UA_f1':res[2],
                                               'UQ_pr':res[3],
                                               'UQ_re':res[4],
                                               'UQ_f1':res[5],
                                               'UD_pr':res[6],
                                               'UD_re':res[7],
                                               'UD_f1':res[8]}
                                        grid.append(tot,ignore_index=True)
    return grid

In [0]:
normalize = [-1,0,1]
variant = [1,2,3]
activation = ['tanh','sigmoid','relu','prelu','lrelu','elu']
units = [16,32,64,128]
dropout = [0.0,0.1,0.2]
regularize = ['l1','l2','l1_l2']
regu_value = [0.1,0.001]
learning_rate = [0.001,0.005,0.01,0.05]
optimizer = ['adam','sgd','adadelta','adagrad','rmsprop']

grid = pd.DataFrame([],columns=['variant','normalize','activation','units','dropout','regularizer',
                                'regularizer_value','learning_rate','optimizer','UA_pr','UA_re','UA_f1',
                                'UQ_pr','UQ_re','UQ_f1','UD_pr','UD_re','UD_f1'])

In [0]:
grid = grid_nn(normalize,variant,activation,units,dropout,regularize,regu_value,learning_rate,optimizer,grid,
           q_train,r_train,s_train,f_train,y_train,q_test1,r_test1,s_test1,f_test1,y_test1,
            q_test2,r_test2,s_test2,f_test2,y_test2,q_test3,r_test3,s_test3,f_test3,y_test3)

NameError: ignored

# Feature Selected

In [0]:
feat_sel = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/Feature Selection.csv")

In [0]:
feat_sel

,Unnamed: 0,"RFC,400,gini","RFC,200,entropy","RFC,200,gini","RFC,400,entropy","RFC,100,gini","GBC,100,0.1","GBC,400,0.01","RFC,300,gini","GBC,300,0.01","GBC,500,0.01"
0,0,Row_no,Row_no,Row_no,Row_no,Row_no,recall,recall,Row_no,recall,recall
1,1,F1_score,F1_score,F1_score,F1_score,F1_score,Row_no,Row_no,recall,Row_no,Row_no
2,2,recall,s_char_count,recall,s_char_count,s_char_count,r_avg_word,r_avg_word,s_char_count,r_char_count,r_avg_word
3,3,s_char_count,recall,s_char_count,recall,recall,r_char_count,r_char_count,F1_score,r_avg_word,r_char_count
4,4,s_avg_word,s_avg_word,s_avg_word,s_avg_word,s_avg_word,q_nouns_vs_words,F1_score,s_avg_word,F1_score,q_nouns_vs_length
5,5,r_char_count,r_char_count,r_char_count,r_char_count,r_char_count,F1_score,q_nouns_vs_length,r_char_count,q_nouns_vs_length,F1_score
6,6,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,s_nouns_vs_length,r_nouns_vs_words,r_nouns,s_nouns_vs_length,r_nouns,r_nouns
7,7,r_avg_word,r_avg_word,r_avg_word,r_avg_word,r_avg_word,s_avg_word,r_word_count,r_avg_word,r_word_count,r_nouns_vs_words
8,8,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,s_verbs_vs_length,q_nouns_vs_length,r_nouns_vs_words,s_verbs_vs_length,r_nouns_vs_words,r_word_count
9,9,precision,precision,precision,precision,precision,r_nouns,s_avg_word,precision,s_avg_word,s_avg_word


In [0]:
def get_top_k_features(feat_sel,k):
    